# Imports and Installation

In [1]:
pip install torch

  Using cached torch-2.3.1-cp311-none-macosx_11_0_arm64.whl.metadata (26 kB)
  Using cached sympy-1.12.1-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached fsspec-2024.6.0-py3-none-any.whl.metadata (11 kB)
  Using cached MarkupSafe-2.1.5-cp311-cp311-macosx_10_9_universal2.whl.metadata (3.0 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached torch-2.3.1-cp311-none-macosx_11_0_arm64.whl (61.0 MB)
Using cached fsspec-2024.6.0-py3-none-any.whl (176 kB)
Using cached jinja2-3.1.4-py3-none-any.whl (133 kB)
Using cached networkx-3.3-py3-none-any.whl (1.7 MB)
Using cached sympy-1.12.1-py3-none-any.whl (5.7 MB)
Using cached MarkupSafe-2.1.5-cp311-cp311-macosx_10_9_universal2.whl (18 kB)
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install qiskit_algorithms

  Using cached qiskit_algorithms-0.3.0-py3-none-any.whl.metadata (4.2 kB)
  Using cached qiskit-1.1.0-cp38-abi3-macosx_11_0_arm64.whl.metadata (12 kB)
  Using cached scipy-1.13.1-cp311-cp311-macosx_12_0_arm64.whl.metadata (60 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 816.6 kB/s eta 0:00:00a 0:00:01
  Using cached rustworkx-0.14.2-cp311-cp311-macosx_11_0_arm64.whl.metadata (10.0 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached stevedore-5.2.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached symengine-0.11.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (1.2 kB)
  Using cached numpy-1.26.4-cp311-cp311-macosx_11_0_arm64.whl.metadata (114 kB)
  Using cached pbr-6.0.0-py2.py3-none-any.whl.metadata (1.3 kB)
Using cached qiskit_algorithms-0.3.0-py3-none-any.whl (308 kB)
Using cached qiskit-1.1.0-cp38-abi3-macosx_11_0_arm64.whl (4.0 MB)
Using cached scipy-1.13.1-cp311-cp311-macosx_12_0_arm64.whl (30.3 MB)
Using cached dill-0.3.8-py3-none-any.whl 

In [3]:
pip uninstall qiskit_nature --yes

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip uninstall pyscf --yes

Note: you may need to restart the kernel to use updated packages.


In [5]:
!pip show qiskit_nature
!pip show pyscf

In [6]:
pip install qiskit-nature

  Using cached qiskit_nature-0.7.2-py3-none-any.whl.metadata (8.0 kB)
  Using cached h5py-3.11.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (2.5 kB)
Using cached qiskit_nature-0.7.2-py3-none-any.whl (2.2 MB)
Using cached h5py-3.11.0-cp311-cp311-macosx_11_0_arm64.whl (2.9 MB)
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install pyscf

  Using cached pyscf-2.6.1-py3-none-macosx_11_0_arm64.whl.metadata (6.3 kB)
Using cached pyscf-2.6.1-py3-none-macosx_11_0_arm64.whl (32.8 MB)
Note: you may need to restart the kernel to use updated packages.


In [8]:
import qiskit
from qiskit.quantum_info import SparsePauliOp, Operator
from qiskit.circuit.library import UnitaryGate,PauliEvolutionGate
import numpy as np
from functools import lru_cache, reduce
from itertools import product
import torch
from qiskit.circuit import Parameter, Gate
from qiskit import QuantumCircuit
from scipy.optimize import minimize
from qiskit.circuit import ParameterVector
from qiskit_algorithms.gradients import ParamShiftEstimatorGradient
from scipy.linalg import expm
from tqdm import tqdm
from qiskit.primitives import Estimator
from qiskit.circuit.library import GlobalPhaseGate, HamiltonianGate
from qiskit_nature.second_q.operators import FermionicOp
import marshal
import csv

# Utils

In [9]:
_pauli_matrices = np.array(
    [[[1, 0], [0, 1]], [[0, 1], [1, 0]], [[0, -1j], [1j, 0]], [[1, 0], [0, -1]]]
)
"""Single-qubit Paulis:    I                 X                   Y                  Z"""

_pauli_letters = "IXYZ"
"""Single-qubit Pauli letters that make up Pauli words."""

@lru_cache
def pauli_basis_matrices(num_qubits):
  if num_qubits < 1:
        raise ValueError(f"Require at least one wire, got {num_qubits}.")
  if num_qubits > 7:
      raise ValueError(
          f"Creating the Pauli basis tensor for more than 7 wires (got {num_qubits}) is deactivated."
      )
  return reduce(np.kron, (_pauli_matrices for _ in range(num_qubits)))[1:]

def detach_numpy(tensor):
    tensor = tensor.detach().cpu()
    if torch._C._functorch.is_gradtrackingtensor(tensor):
        tensor = torch._C._functorch.get_unwrapped(tensor)
        return np.array(tensor.storage().tolist()).reshape(tensor.shape)
    return tensor.numpy()

def pauli_decompose(matrix, num_qubits):
  basis = pauli_basis_matrices(num_qubits)
  coefficients = np.tensordot(torch.tensor(basis), matrix, [[1, 2], [-1, -2]]) * (1 / 2**num_qubits)
  return coefficients

@lru_cache
def pauli_basis_strings(num_wires):
  return ["".join(letters) for letters in product(_pauli_letters, repeat=num_wires)][1:]

def get_file_path(file_name, data_directory):
    if file_name:
        if file_name[-5:] != '.data':
            file_name = file_name + ".data"
    else:
        raise ("File name is not provided.")

    file_path = data_directory + '/' + file_name

    return file_path
sign_arr = ['+', '-']
def load_operator(file_name=None, data_directory=None, plain_text=False, nqubits=4):
    """Load FermionOperator or QubitOperator from file.

    Args:
        file_name: The name of the saved file.
        data_directory: Optional data directory to change from default data
                        directory specified in config file.
        plain_text: Whether the input file is plain text

    Returns:
        operator: The stored FermionOperator, BosonOperator,
            QuadOperator, or QubitOperator

    Raises:
        TypeError: Operator of invalid type.
    """
    file_path = get_file_path(file_name, data_directory)
    if plain_text:
        with open(file_path, 'r') as f:
            data = f.read()
            operator_type, operator_terms = data.split(":\n")
            operator = FermionicOp(operator_terms)
    else:
        with open(file_path, 'rb') as f:
            data = marshal.load(f)
            operator_type = data[0]
            operator_terms = data[1]
        if operator_type == 'FermionOperator':
            max_term = 0
            op_dict = {}
            for term, coeff in operator_terms.items():
                op = ""
                for term_i in term:
                  val, sign = term_i[0], term_i[1]
                  op += sign_arr[sign] + "_" + str((nqubits-1-val)) + " "
                  max_term = max(val, max_term)
                op_dict[op] =  coeff

            operator = FermionicOp(op_dict, max_term+1, copy=False)
    return operator

def problem_hamiltonian(n_qubits: int, seed: int, hamiltonian_directory: str):
    print(f"{str(n_qubits).zfill(2)}qubits_{str(seed).zfill(2)}")
    ham = load_operator(
        file_name=f"{str(n_qubits).zfill(2)}qubits_{str(seed).zfill(2)}",
        data_directory=hamiltonian_directory,
        plain_text=False,
        nqubits=n_qubits,
    )
    return ham

# SU Implementation

In [10]:
class SU(Gate):
    def __init__(self, theta:np.ndarray|list[Parameter], nqubits, label=None):
        if isinstance(theta, np.ndarray):
            self.theta = torch.from_numpy(np.array(theta))
        super().__init__('SU', nqubits, theta, label=label)

    def _define(self):
        qc = QuantumCircuit(self.num_qubits)
        qc.unitary(self.to_matrix(), range(self.num_qubits))

        self.definition = qc

    def to_matrix(self):
        num_qubits = self.num_qubits
        mat = self.compute_matrix(self.theta, num_qubits)
        return detach_numpy(mat)

    @staticmethod
    def compute_matrix(theta, num_qubits):
      if num_qubits > 5:
        matrices = product(_pauli_matrices, repeat=num_qubits)
        # Drop the identity from the generator of matrices
        _ = next(matrices)
        A = sum(t * reduce(np.kron, letters) for t, letters in zip(theta, matrices))
      else:
        paulis = torch.from_numpy(pauli_basis_matrices(num_qubits))
        theta = theta.type(torch.complex128)
        A = torch.tensordot(theta, paulis, dims=[[-1], [0]])

      return torch.matrix_exp(torch.mul(A, 1j))

    def get_one_parameter_generators(self):
      theta = self.theta
      num_qubits = self.num_qubits

      def split_matrix(theta):
        mat = self.compute_matrix(theta, num_qubits)
        return torch.real(mat), torch.imag(mat)

      rjac, ijac = torch.autograd.functional.jacobian(split_matrix, theta)
      jac = rjac + 1j * ijac

      U_dagger = self.compute_matrix(torch.mul(theta, -1), num_qubits)

      # U_dagger = U_dagger.type(torch.complex128)
      # jac = detach_numpy(jac)
      jac = jac.type(torch.complex128)
      U_dagger = U_dagger.type(torch.complex128)
      ujac = torch.tensordot(U_dagger, jac, dims=([1], [0]))
      ujac = ujac.permute(2, 0, 1)
      return detach_numpy(ujac)

      # ujac = np.tensordot(U_dagger, jac, axes=([1], [0]))
      # ujac = np.transpose(ujac, [2, 0, 1])
      # return ujac

    def get_one_parameter_coeffs(self):
      num_qubits = self.num_qubits
      theta = self.theta
      generators = self.get_one_parameter_generators()
      Omega = pauli_decompose(generators, num_qubits)

      # skew matrix condition on Omega
      return Omega

def su4(params, nqubits):
    return SU(params, nqubits)

In [11]:
def apply_su4_in_layers(theta, depth, nqubits, fix_pars_per_layer=False):
    """Apply a (callable) operation in layers.

    Args:
        theta (tensor_like): The arguments passed to the operations. The expected shape
            is ``(depth, k, num_params_op)``, where ``k`` is determined by ``fix_pars_per_layer``
            and ``num_params_op`` is the number of paramters each operation takes.
        Op (callable): The operation to apply
        depth (int): The number of layers to apply
        fix_pars_per_layer (bool): Whether or not all operations applied in parallel share the
            same parameters. If ``True``, the dimension ``k`` in the shape of ``theta`` is set to
            two, otherwise it is set to ``nqubits``.

    """
    qc = qiskit.QuantumCircuit(nqubits)
    qubits = list(range(nqubits))

    if fix_pars_per_layer:
        size = 2
    else:
        size = nqubits

    for d in range(depth):
        # Even-odd qubit pairs
        idx = 0
        for q in qubits[0::2]:
            su = su4(theta[(d*size + idx)*15:(d*size + idx +1)*15], 2)
            qc = qc.compose(su, [q, (q+1+d)%nqubits])
            if q == nqubits - 2:
                idx += 1
            elif not fix_pars_per_layer:
                idx += 1

        # Odd-even qubit pairs
        for q in qubits[1::2]:
            su = su4(theta[(d*size + idx)*15:(d*size + idx +1)*15],2)
            qc = qc.compose(su, [q, (q+1+d)%nqubits])
            if not fix_pars_per_layer:
                idx += 1
    return qc

# Grad function

# Read observable

In [12]:
from qiskit_nature.second_q.hamiltonians import FermiHubbardModel, HeisenbergModel
from qiskit_nature.second_q.hamiltonians.lattices import BoundaryCondition,LineLattice
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.hamiltonians import QuadraticHamiltonian

#read observable from file
def make_observable(num_qubits, seed=0, mat=True):
  hamiltonian_directory = "."
  print("MN = ", num_qubits)
  ham = problem_hamiltonian(num_qubits, seed, hamiltonian_directory)
  H = JordanWignerMapper().map(ham)
  if mat:
    H_jw = H.to_matrix()
    return H, H_jw
  else:
    return H

# VQE

In [13]:
def print_cpu_usage():
  import psutil
  split_bar = '='*20
  memory_info = psutil.virtual_memory()._asdict()
  print(f"{split_bar} Memory Usage {split_bar}")
  for k,v in memory_info.items():
    print(k, v)
  print(f"{split_bar} CPU Usage {split_bar}")
  print(f"CPU percent: {psutil.cpu_percent()}%")

In [14]:
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit_algorithms.gradients import FiniteDiffEstimatorGradient
from qiskit_algorithms import VQE
import math

def generate_init_params(num_qubits, num_layers):
  """
  Generate initial parameters for the ansatz.
  Returns:
      list[float]: The initial parameters for the ansatz.
  """
  NUM_WIRES = num_qubits
  LAYERS_PER_BLOCK = 15*num_qubits

  init_params = np.random.uniform(
      0, 2 * np.pi, LAYERS_PER_BLOCK * num_layers
  )
  for i in range(num_layers):
      layer_param_sum = np.sum(
          init_params[i * LAYERS_PER_BLOCK : (i + 1) * LAYERS_PER_BLOCK]
      )
      init_params[i * LAYERS_PER_BLOCK : (i + 1) * LAYERS_PER_BLOCK] *= (
          math.pi / (2 * NUM_WIRES) / layer_param_sum
      )

  return init_params


def vqe(
    num_qubits,
    observable,
    depth,
    optimizer,
    max_steps,
    init_param=None):

    # n = 4**num_SUgate_qubits - 1 = 15
    param_len = depth*num_qubits*15

    estimator = Estimator()

    # List to store the minimized values after every iteration
    minimized_values = []

    def cost_function(params):
        """Cost function of the VQE."""

        # Initialize Hartree-Fock state
        num_orbitals = num_qubits // 2
        num_particles = num_qubits // 2
        num_alpha = num_particles // 2
        num_beta = num_particles // 2

        mapper = JordanWignerMapper()
        init_state = HartreeFock(num_orbitals, (num_alpha, num_beta) , qubit_mapper=mapper)

        # Setup the variational form for UCCSD (CISD equivalent in quantum chemistry)
        '''
        var_form = UCCSD(
            num_orbitals,
            (num_alpha, num_beta),
            mapper,
            initial_state=init_state,
        )
        '''
        circ = init_state.compose(apply_su4_in_layers(params, depth, num_qubits, fix_pars_per_layer=False))


        estimator_expvals = estimator.run(circuits=circ, observables=observable).result().values[0]

        # Store the minimized value
        #minimized_values.append(estimator_expvals)
        #print("Minimized Exp Value: ", estimator_expvals)
        #print("-----------------------------------------")

        return estimator_expvals

    def grad_function(params):
        """ use FiniteDiffEstimatorGradient """

        # Initialize Hartree-Fock state
        num_orbitals = num_qubits // 2
        num_particles = num_qubits // 2
        num_alpha = num_particles // 2
        num_beta = num_particles // 2

        mapper = JordanWignerMapper()
        init_state = HartreeFock(num_orbitals, (num_alpha, num_beta) , qubit_mapper=mapper)

        # Setup the variational form for UCCSD (CISD equivalent in quantum chemistry)
        '''
        var_form = UCCSD(
            num_orbitals,
            (num_alpha, num_beta),
            mapper,
            initial_state=init_state,
        )
        '''

        # This could be optimized
        indices = [*range(len(params))]

        offset = np.identity(len(params))[indices, :]
        plus = params + 0.001 * offset
        minus = params - 0.001 * offset

        # length of circ_list is param_len
        circ_plus_list = [init_state.compose(apply_su4_in_layers(params_value, depth, num_qubits, fix_pars_per_layer=False)) for params_value in plus]
        circ_minus_list = [init_state.compose(apply_su4_in_layers(params_value, depth, num_qubits, fix_pars_per_layer=False)) for params_value in minus]

        job_plus = estimator.run(circ_plus_list, [observable] * param_len)
        job_minus = estimator.run(circ_minus_list, [observable] * param_len)
        results_plus = job_plus.result()
        results_minus = job_minus.result()


        return (results_plus.values-results_minus.values)/0.002
        #return circ_plus_list


    # Create initial parameters
    if init_param is None:
        init_params = np.array([0.1]*param_len)
    else:
        init_params = init_param

    Optimizer = optimizer(maxiter=max_steps)
    # Optimizer = optimizer(maxfun=max_steps)
    opt_result = Optimizer.minimize(fun=cost_function, x0=init_params, jac=grad_function)

    return opt_result

In [15]:
def separat_vqe(
    num_qubits,
    observable,
    depth,
    optimizer,
    max_steps,
    separat_steps,
    init_param=None):

    with open("vqe_params_" + datetime + ".csv", "a", newline='') as fd:
        statewriter = csv.writer(fd, delimiter=',', quoting=csv.QUOTE_MINIMAL)
        if init_param is None:
            result = vqe(num_qubits,
                         observable,
                         depth,
                         optimizer,
                         max_steps=separat_steps,
                         init_param=None)
        else:
            result = vqe(num_qubits,
                         observable,
                         depth,
                         optimizer,
                         max_steps=separat_steps,
                         init_param=init_param)
        print(result.fun)
        statewriter.writerow(result.x)


        for i in range(separat_steps,max_steps,separat_steps):
            result = vqe(num_qubits,
                         observable,
                         depth,
                         optimizer,
                         separat_steps,
                         init_param=result.x)
            print(result.fun)
            statewriter.writerow(result.x)
            print("completed steps:",i)



In [16]:
def vqe_full(
    num_qubits,
    observable,
    depth,
    optimizer,
    max_steps,
    skip_steps,
    starting_param=None):

  for i in range(skip_steps,max_steps,skip_steps):
    #run for 1 step
    result = vqe(num_qubits,
                  observable,
                  depth,
                  optimizer,
                  skip_steps,
                  init_param=starting_param)
    print(result.fun)
    # save init parms

    with open("vqe_params_" + str(i) + ".csv", "w", newline='') as fd:
      statewriter = csv.writer(fd, delimiter=',', quoting=csv.QUOTE_MINIMAL)
      starting_params = result.x
      statewriter.writerow(result.x)

    print("completed steps:",i)



In [22]:
nqubits = 12  # Number of qubits
fix_pars_per_layer = False
observable = make_observable(nqubits, mat=False)
# observable

MN =  12
12qubits_00


In [23]:
print_cpu_usage()

==================== Memory Usage ====================
total 8589934592
available 1320960000
percent 84.6
used 3265282048
free 43958272
active 1293746176
inactive 1200652288
wired 1971535872
==================== CPU Usage ====================
CPU percent: 78.5%


In [24]:
# from qiskit_algorithms.optimizers import SLSQP, P_BFGS, NELDER_MEAD
# import time

# start = time.time()
# testt = vqe([0],
#             nqubits,
#             observable,
#             [],
#             depth=2,
#             optimizer=NELDER_MEAD,
#             max_steps=100
#            )
# # circuit.num_parameters
# end = time.time()
# print("Time Taken: ", end-start)

In [25]:
# SLSQP 100
# testt.fun

In [26]:
from qiskit_algorithms.optimizers import SLSQP, P_BFGS, NELDER_MEAD
import time

start = time.time()
datetime = time.strftime("%Y-%m-%d_%H-%M-%S")

testt = vqe_full(
    nqubits,
    observable,
    depth=1,
    optimizer=SLSQP,
    max_steps=10,
    skip_steps=2,
    starting_param=None)

# For running vqe without saving params 
'''
test = vqe(nqubits,
           observable,
           depth=1,
           optimizer=SLSQP,
           max_steps=10,
           init_param=None)
'''
end = time.time()
print("Time Taken: ", end-start)

-8.284832195943544
completed steps: 2
-8.284832195943544
completed steps: 4
-8.284832195943544
completed steps: 6
-8.284832195943544
completed steps: 8
Time Taken:  146.6697609424591


In [22]:
testt.fun

AttributeError: 'NoneType' object has no attribute 'fun'

In [ ]:
energies = np.linalg.eigvalsh(observable_matrix)
gs_energy = energies[0]
gs_energy

TODO:

1. Optimizer Table for complexity


In [ ]:
#2. check mps simulator